### This code is a new branch where CSAT data that use is after analysis (Taking Out The Unnecessary Data)

In [39]:
import gspread
import pandas as pd
from datetime import date
from google.oauth2.service_account import Credentials

## Pipeline A

In [40]:
SERVICE_ACCOUNT_FILE = '../../key/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

sheet = client.open("Chatbot Ops Report").worksheet("[CSAT After Take Out]")
data = sheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [41]:
# Copying the DF
df_clean = (
    df.copy()
    .iloc[:, [0,5,6]]
    .drop_duplicates(subset='Date')
)

# Changing the Date data type
df_clean['Date'] = pd.to_datetime(
    df_clean['Date'], format='%d-%b-%Y', errors='coerce'
)

# dropping NA
df_clean = df_clean.dropna(subset=['Date'])

df_clean

,Date,CSAT [Before],CSAT [After]
0,2024-03-01,3.32,3.68
1,2024-03-02,3.31,3.67
2,2024-03-03,3.80,3.95
3,2024-03-04,3.62,3.97
4,2024-03-05,3.52,3.65
...,...,...,...
537,2025-08-28,2.68,3.14
538,2025-08-29,3.18,3.54
539,2025-08-30,3.29,3.82
540,2025-08-31,3.36,3.95


## Pipeline B

In [42]:
df_b = pd.read_csv('../../dataset_kula/csat/akulaku_superset_presto_hanmm_ozy.prazuganda_1755663318_8_19.csv')
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11949 entries, 0 to 11948
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   business                           11949 non-null  object 
 1   id                                 11949 non-null  object 
 2   currentAccount                     11949 non-null  object 
 3   userId                             11949 non-null  int64  
 4   shopid                             11949 non-null  int64  
 5   summary_type                       0 non-null      float64
 6   summary_classify                   0 non-null      float64
 7   ticketid                           4805 non-null   object 
 8   Category 1                         4492 non-null   object 
 9   Category 2                         4146 non-null   object 
 10  Category 3                         3912 non-null   object 
 11  summary_text                       0 non-null      flo

In [43]:
# Filtering the currentAccount into robot only
df_filtered = df_b[df_b['currentAccount'] == '3-1-robot']

# Cleaning and Counting
score = df_filtered['score'].dropna()

total_responden = score.count()
total_rating = int(score.sum())
csat = round(total_rating / total_responden, 2) if total_responden > 0 else 0

# Good survey and bad survey counting
good_survey = score[score >= 3].count()
bad_survey = score[score < 3].count()

In [44]:
"""
-- Not necessary at the moment --

# Checking if the yesterday data already exists in the DataFrame
yesterday = pd.Timestamp(date.today() - pd.Timedelta(days=1))

if (df_clean['Date'].iloc[-1].normalize != yesterday.normalize()):
    new_row = pd.DataFrame([{
        'Date': yesterday,
        'Total Good Survey': good_survey,
        'Total Bad Survey': bad_survey,
        'Total Responden': total_responden,
        'Total Rating': total_rating,
        'CSAT': csat
    }])

    df_clean = pd.concat([df_clean, new_row], ignore_index=True)

"""

"\n-- Not necessary at the moment --\n\n# Checking if the yesterday data already exists in the DataFrame\nyesterday = pd.Timestamp(date.today() - pd.Timedelta(days=1))\n\nif (df_clean['Date'].iloc[-1].normalize != yesterday.normalize()):\n    new_row = pd.DataFrame([{\n        'Date': yesterday,\n        'Total Good Survey': good_survey,\n        'Total Bad Survey': bad_survey,\n        'Total Responden': total_responden,\n        'Total Rating': total_rating,\n        'CSAT': csat\n    }])\n\n    df_clean = pd.concat([df_clean, new_row], ignore_index=True)\n\n"

In [45]:
df_clean.to_csv('../../dataset_kula/csat_takeout.csv')